In [43]:
import os
import numpy as np
import pickle
import math
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [81]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./image_data/', 
                                        train=True,
                                        download=True, 
                                        transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=100,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./image_data', 
                                       train=False,
                                       download=True, 
                                       transform=transform)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=1,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')
# classes = ('plane', 'car')

Files already downloaded and verified
Files already downloaded and verified


In [53]:
class neural_network_50(nn.Module):
    
    def __init__(self):
        super(neural_network_50, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=60, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(120 * 4 * 4, 64)
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x):        
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 120 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x



In [55]:
my_network = neural_network_50()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(my_network.parameters(), lr=0.1)
# optimizer = optim.SGD(my_network.parameters(), lr=0.1)

model_directory_path = './PyTorch_model'
model_path = model_directory_path + '/cifar10_50_percent.pt'

if not os.path.exists(model_directory_path):
    os.makedirs(model_directory_path)

# if os.path.isfile(model_path):
#     my_network.load_state_dict(torch.load(model_path))
#     print('Pre Trained model exist, Loading')
# else:

train_continue = True
early_stopping = np.zeros(3)
count = 0
previous_loss = float('Inf')

while(train_continue==True):

    training_loss = 0.0
    testing_loss = 0.0
    total_correct_test = 0
    total_image_test = 0

    for train_data in trainloader:
        input_batch, label_batch = train_data
        input_batch = input_batch[label_batch<2]
        if np.array(input_batch).shape[0] == 0:
            continue
        label_batch = label_batch[label_batch<2]
        optimizer.zero_grad()
        output_batch = my_network(input_batch)
        loss = criterion(output_batch, label_batch)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()

    print('Tranining of Epoch:', count,' Done')

    with torch.no_grad():
        for test_data_batch in testloader:
            input_batch, label_batch = test_data_batch
            input_batch = input_batch[label_batch<2]
            if np.array(input_batch).shape[0] == 0:
                continue
            label_batch = label_batch[label_batch<2]
            output_batch = my_network(input_batch)
            loss = criterion(output_batch, label_batch)
            testing_loss += loss.item()

            _, predicted = torch.max(output_batch.data, 1)
            total_image_test += label_batch.size(0)
            total_correct_test += (predicted == label_batch).sum().item()
        test_accuracy = round(total_correct_test / total_image_test * 100,3)

    if testing_loss<previous_loss:
        previous_loss = testing_loss
        torch.save(my_network.state_dict(), model_path)  

    if (testing_loss-previous_loss)>=0.01:
        early_stopping[count%3] = 1
    else:
        early_stopping[count%3] = 0

    count += 1

    if np.sum(early_stopping) == 3:
        train_continue = False

    print('epoch:',count ,', Train Loss:', round(training_loss,3),', Test Loss:', round(testing_loss,3),', Test Accuracy:', round(test_accuracy,3),'%')

Tranining of Epoch: 0  Done
epoch: 1 , Train Loss: 268.21 , Test Loss: 29.983 , Test Accuracy: 87.7 %
Tranining of Epoch: 1  Done
epoch: 2 , Train Loss: 126.086 , Test Loss: 19.208 , Test Accuracy: 91.65 %
Tranining of Epoch: 2  Done
epoch: 3 , Train Loss: 82.421 , Test Loss: 14.273 , Test Accuracy: 94.4 %


KeyboardInterrupt: 

In [91]:
my_network = neural_network_50()

model_directory_path = './PyTorch_model'
model_path = model_directory_path + '/cifar10_50_percent.pt'

my_network.load_state_dict(torch.load(model_path))

with torch.no_grad():
    for test_data_batch in testloader:
        input_batch, label_batch = test_data_batch
        input_batch = input_batch[label_batch<2]
        if np.array(input_batch).shape[0] == 0:
            continue
        label_batch = label_batch[label_batch<2]
        output_batch = my_network(input_batch)
        _, predicted = torch.max(output_batch.data, 1)
        total_image_test += label_batch.size(0)
        total_correct_test += (predicted == label_batch).sum().item()
    test_accuracy = round(total_correct_test / total_image_test * 100,3)

    print('Accuracy:',test_accuracy)

Accuracy: 90.403


In [88]:
class neural_network_25(nn.Module):
    
    def __init__(self):
        super(neural_network_25, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=30, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(60 * 4 * 4, 64)
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x):        
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 60 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [60]:
my_network = neural_network_25()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(my_network.parameters(), lr=0.0001)
# optimizer = optim.Adam(my_network.parameters(), lr=0.0001)

model_directory_path = './PyTorch_model'
model_path = model_directory_path + '/cifar10_25_percent.pt'

if not os.path.exists(model_directory_path):
    os.makedirs(model_directory_path)

# if os.path.isfile(model_path):
#     my_network.load_state_dict(torch.load(model_path))
#     print('Pre Trained model exist, Loading')
# else:

train_continue = True
early_stopping = np.zeros(3)
count = 0
previous_loss = float('Inf')

while(train_continue==True):

    training_loss = 0.0
    testing_loss = 0.0
    total_correct_test = 0
    total_image_test = 0

    for train_data in trainloader:
        input_batch, label_batch = train_data
        input_batch = input_batch[label_batch<2]
        if np.array(input_batch).shape[0] == 0:
            continue
        label_batch = label_batch[label_batch<2]
        optimizer.zero_grad()
        output_batch = my_network(input_batch)
        loss = criterion(output_batch, label_batch)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()

    print('Tranining of Epoch:', count,' Done')

    with torch.no_grad():
        for test_data_batch in testloader:
            input_batch, label_batch = test_data_batch
            input_batch = input_batch[label_batch<2]
            if np.array(input_batch).shape[0] == 0:
                continue
            label_batch = label_batch[label_batch<2]
            output_batch = my_network(input_batch)
            loss = criterion(output_batch, label_batch)
            testing_loss += loss.item()

            _, predicted = torch.max(output_batch.data, 1)
            total_image_test += label_batch.size(0)
            total_correct_test += (predicted == label_batch).sum().item()
        test_accuracy = round(total_correct_test / total_image_test * 100,3)

    if testing_loss<previous_loss:
        previous_loss = testing_loss
        torch.save(my_network.state_dict(), model_path)  

    if (testing_loss-previous_loss)>=0.01:
        early_stopping[count%3] = 1
    else:
        early_stopping[count%3] = 0

    count += 1

    if np.sum(early_stopping) == 3:
        train_continue = False

    print('epoch:',count ,', Train Loss:', round(training_loss,3),', Test Loss:', round(testing_loss,3),', Test Accuracy:', round(test_accuracy,3),'%')

Tranining of Epoch: 0  Done
epoch: 1 , Train Loss: 257.198 , Test Loss: 35.257 , Test Accuracy: 85.2 %
Tranining of Epoch: 1  Done
epoch: 2 , Train Loss: 181.108 , Test Loss: 32.341 , Test Accuracy: 86.15 %
Tranining of Epoch: 2  Done
epoch: 3 , Train Loss: 162.614 , Test Loss: 28.592 , Test Accuracy: 88.25 %


KeyboardInterrupt: 

In [90]:
my_network = neural_network_25()

model_directory_path = './PyTorch_model'
model_path = model_directory_path + '/cifar10_25_percent.pt'

my_network.load_state_dict(torch.load(model_path))
with torch.no_grad():
    
    for test_data_batch in testloader:
        input_batch, label_batch = test_data_batch
        input_batch = input_batch[label_batch<2]
        if np.array(input_batch).shape[0] == 0:
            continue
        label_batch = label_batch[label_batch<2]
        output_batch = my_network(input_batch)
        _, predicted = torch.max(output_batch.data, 1)
        total_image_test += label_batch.size(0)
        total_correct_test += (predicted == label_batch).sum().item()
    test_accuracy = round(total_correct_test / total_image_test * 100,3)
    print('Accuracy:',test_accuracy)

Accuracy: 90.192


In [63]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [64]:
count_parameters(my_network)

80904